## Projeto de Sinais e Sistemas Dinâmicos
### Detecção de faces utilizando a base ORL Faces
Aluno: Epitácio Pessoa de Brito Neto

Professor: Derzu Omaia

## 1. Introdução

Este é o projeto da disciplina de Sinais e Sistemas Dinâmicos do curso de Engenharia da Computação - UFPB, ministrada pelo professor Derzu Omaia. O intuito deste projeto é a obtenção de conhecimentos de aplicação de assuntos da disciplina em nossa área, neste caso, temos o auxilio da transformada de Fourier para reconhecimento facial, utilizando a base de dados disponibilizada pelo professor, e utilizaremos como apoio os nossos conhecimentos de programação, análise de pré-processamento de dados e Inteligência Artificial. 

#### Cabeçalho dos includes

In [1]:
import pandas as pd
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cv2
import sys
import math
from sklearn.metrics import confusion_matrix, accuracy_score, mean_squared_error
from scipy.spatial import distance
import cmath
from numpy import linalg as LA
import scipy.spatial as spt

## 2. Declaração de funções de auxilio

Primeiramente definimos algumas funções para auxiliar futuramente na análise. A primeira delas é para diminuir/aumentar a resolução para fazermos a análise de cada uma delas. 

In [2]:
def cortar_imagem(n, imagem):    
    largura_da_imagem = 92
    altura_da_imagem = 112
    
    x = altura_da_imagem//2
    y = largura_da_imagem//2
    r = n//2
    
    impar = 0
    if n % 2 != 0:
        impar = -1
    
    return imagem[x-r+impar:x+r,y-r+impar:y+r]

As próximas funções são, basicamente, o algoritmo do KNN, o suporte das bibliotecas como SKLearn foi tentada para fazer, mas se tornou mais prático fazer o projeto a partir do próprio algoritmo. Nesta primeira função, utilizamos dela para resolver 4 dos 5 casos de teste que foi requisitado: análise apenas da parte real, apenas da parte imaginária, a soma do real com a imaginária, e um 'merge' do real com imaginário.

In [3]:
def knn(subjects, dataframe, mode):
    
    nearest_indexes = []
    nearest_labels = []
    
    for i, labeless_image in enumerate(subjects):
        
        nearest = math.inf
        nearest_index = 0
        
        for j, labeled_image in enumerate(dataframe):
            
            if mode == 'r':
                mse = mean_squared_error(labeled_image.real, labeless_image.real)
            if mode == 'i':
                mse = mean_squared_error(labeled_image.imag, labeless_image.imag)
            if mode == 'ri':
                mse_real = mean_squared_error(labeled_image.real, labeless_image.real)
                mse_imag = mean_squared_error(labeled_image.imag, labeless_image.imag)
                mse = mse_real + mse_imag
            if mode == 'rim':
                labeled_image_aux = labeled_image
                labeled_image_aux = eucld(labeled_image_aux.real, labeled_image_aux.imag)
                labeless_image = eucld(labeless_image.real, labeless_image.imag)
                mse = mean_squared_error(labeled_image_aux, labeless_image)
            if mode == 'rimul':
                mse_real = mean_squared_error(labeled_image.real, labeless_image.real)
                mse_imag = mean_squared_error(labeled_image.imag, labeless_image.imag)
                mse = np.dot(mse_real, mse_imag)
            if mse < nearest:
                nearest = mse
                nearest_index = j
        nearest_indexes.append(nearest_index)
        nearest_labels.append(labels[nearest_index])
        
    return nearest_labels

O KNN2 foi feito especificamente para resolver o caso de teste 'Real e Imaginario', já que mudaria a estrutura que foi feita para resolver os outros 4 casos de teste.

In [4]:
def knn2(subjects, dataframe):
    
    nearest_indexes = []
    nearest_labels = []
    
    for i, labeless_image in enumerate(subjects):
        
        nearest = math.inf
        nearest_index = 0
        
        for j, labeled_image in enumerate(dataframe):
            mse_real_imag = mean_squared_error(labeled_image.real, labeless_image.imag)
            mse_imag_real = mean_squared_error(labeled_image.imag, labeless_image.real)
            mse_real = mean_squared_error(labeled_image.real, labeless_image.real)
            mse_imag = mean_squared_error(labeled_image.imag, labeless_image.imag)

            distance_list = [mse_real_imag, mse_imag_real, mse_real, mse_imag]
            mse = sorted(distance_list)[0]
            if mse < nearest:
                nearest = mse
                nearest_index = j
        nearest_indexes.append(nearest_index)
        nearest_labels.append(labels[nearest_index])
        
    return nearest_labels

Foi tentado criar uma função para distancia euclidiana para matrizes Numpy, mas não foi conseguido êxito, o que resultou na falha ao fazer o teste de 'merge'.

In [5]:
def eucld(subject, dataframe):
    V = spt.distance.pdist(subject.T, 'sqeuclidean')
    return spt.distance.squareform(V)

## 3. Pré-processamento dos dados

Aqui armazenamos e transformamos as imagens em Numpy arrays

In [6]:
quantidade_de_pessoas = 40
fotos_pessoas = 10

fotos_das_pessoas = list() 
primeira_foto_das_pessoas = list() 
fotos_de_cada_pessoa = list()

for i in range(1, quantidade_de_pessoas+1):
    for j in range(1, fotos_pessoas+1):
        fotos_das_pessoas.append(np.array(cv2.imread(f'.\orl_faces\s{i}\{j}.pgm',0)))
        if j == 1:
            primeira_foto_das_pessoas.append(np.array(cv2.imread(f'.\orl_faces\s{i}\{j}.pgm',0)))
        if j != 1:
            fotos_de_cada_pessoa.append(np.array(cv2.imread(f'.\orl_faces\s{i}\{j}.pgm',0)))

O tamanho do dataframe bate com o esperado, dado as informações contidas na descrição do projeto.

In [7]:
print(f"Tamanho: {len(fotos_de_cada_pessoa)}")

Tamanho: 360


Aqui é feita a transformada de Fourier dos arrays que fizemos anteriormente, em seguida fazemos um shift para centralizar o que conseguimos transformar, para facilitar a manipulação durante a análise.

In [8]:
pessoas_fft = [np.fft.fft2(pessoa) for pessoa in fotos_de_cada_pessoa]

pessoa_fft = [np.fft.fft2(pessoa) for pessoa in primeira_foto_das_pessoas]

In [9]:
pessoas_fft_shift = [np.fft.fftshift(pessoa) for pessoa in pessoas_fft]

pessoa_fft_shift = [np.fft.fftshift(pessoa) for pessoa in pessoa_fft]

Aqui ajustamos a resolução das imagens originais, os testes serão feitos com valores de resolução entre [2-30], intervalo que considerei satisfatível, dado os valores de acurácia que vão ser vistos em breve. 

In [10]:
dimensao = 30
corte = []
corte_unico = []
for imagem in pessoas_fft_shift:
    corte.append(np.array(cortar_imagem(dimensao, imagem)).flatten())
      
for imagem in pessoa_fft_shift:
    corte_unico.append(np.array(cortar_imagem(dimensao, imagem)).flatten())
    
X = np.array(corte)

In [11]:
labels = [i for i in range(40) for j in range(9)]

## 4. Treinamento
### 4.1 Caso de teste: Real

In [12]:
dimensoes = [i for i in range(2,31)]

labels_predict_real = []

print("Carregando")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_real.append(knn(subject, dataframe, 'r'))
print("OK!")

Carregando
OK!


In [13]:
y_true = [i for i in range(0,40)]

for i, y_pred in enumerate(labels_predict_real):
    print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))

Acurácia para a resolução  2: 0.9
Acurácia para a resolução  3: 0.975
Acurácia para a resolução  4: 0.975
Acurácia para a resolução  5: 0.975
Acurácia para a resolução  6: 0.975
Acurácia para a resolução  7: 0.975
Acurácia para a resolução  8: 1.0
Acurácia para a resolução  9: 1.0
Acurácia para a resolução 10: 1.0
Acurácia para a resolução 11: 1.0
Acurácia para a resolução 12: 1.0
Acurácia para a resolução 13: 1.0
Acurácia para a resolução 14: 1.0
Acurácia para a resolução 15: 1.0
Acurácia para a resolução 16: 1.0
Acurácia para a resolução 17: 1.0
Acurácia para a resolução 18: 1.0
Acurácia para a resolução 19: 1.0
Acurácia para a resolução 20: 1.0
Acurácia para a resolução 21: 1.0
Acurácia para a resolução 22: 1.0
Acurácia para a resolução 23: 1.0
Acurácia para a resolução 24: 1.0
Acurácia para a resolução 25: 1.0
Acurácia para a resolução 26: 1.0
Acurácia para a resolução 27: 1.0
Acurácia para a resolução 28: 1.0
Acurácia para a resolução 29: 1.0
Acurácia para a resolução 30: 1.0


### 4.2 Caso de teste: Imaginario

In [14]:
dimensoes = [i for i in range(2,31)]

labels_predict_imag = []

print("Carregando")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_imag.append(knn(subject, dataframe, 'i'))
print("OK!")

Carregando
OK!


In [15]:
y_true = [i for i in range(0,40)]

for i, y_pred in enumerate(labels_predict_imag):
    print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))

Acurácia para a resolução  2: 0.5
Acurácia para a resolução  3: 0.875
Acurácia para a resolução  4: 0.925
Acurácia para a resolução  5: 0.925
Acurácia para a resolução  6: 0.925
Acurácia para a resolução  7: 0.95
Acurácia para a resolução  8: 0.95
Acurácia para a resolução  9: 0.95
Acurácia para a resolução 10: 0.95
Acurácia para a resolução 11: 0.95
Acurácia para a resolução 12: 0.95
Acurácia para a resolução 13: 0.95
Acurácia para a resolução 14: 0.95
Acurácia para a resolução 15: 0.95
Acurácia para a resolução 16: 0.95
Acurácia para a resolução 17: 0.95
Acurácia para a resolução 18: 0.95
Acurácia para a resolução 19: 0.95
Acurácia para a resolução 20: 0.95
Acurácia para a resolução 21: 0.95
Acurácia para a resolução 22: 0.95
Acurácia para a resolução 23: 0.95
Acurácia para a resolução 24: 0.95
Acurácia para a resolução 25: 0.95
Acurácia para a resolução 26: 0.95
Acurácia para a resolução 27: 0.95
Acurácia para a resolução 28: 0.95
Acurácia para a resolução 29: 0.95
Acurácia para a r

### 4.3 Caso de teste: Real + Imaginario 

In [16]:
dimensoes = [i for i in range(2,31)]

labels_predict_real_imag = []

print("Carregando")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_real_imag.append(knn(subject, dataframe, 'ri'))
print("OK!")

Carregando
OK!


In [17]:
y_true = [i for i in range(0,40)]

for i, y_pred in enumerate(labels_predict_real_imag):
    print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))

Acurácia para a resolução  2: 0.975
Acurácia para a resolução  3: 0.975
Acurácia para a resolução  4: 0.975
Acurácia para a resolução  5: 0.95
Acurácia para a resolução  6: 0.95
Acurácia para a resolução  7: 0.95
Acurácia para a resolução  8: 0.95
Acurácia para a resolução  9: 0.95
Acurácia para a resolução 10: 0.95
Acurácia para a resolução 11: 0.95
Acurácia para a resolução 12: 0.95
Acurácia para a resolução 13: 0.95
Acurácia para a resolução 14: 0.95
Acurácia para a resolução 15: 0.95
Acurácia para a resolução 16: 0.95
Acurácia para a resolução 17: 0.95
Acurácia para a resolução 18: 0.95
Acurácia para a resolução 19: 0.95
Acurácia para a resolução 20: 0.95
Acurácia para a resolução 21: 0.95
Acurácia para a resolução 22: 0.95
Acurácia para a resolução 23: 0.95
Acurácia para a resolução 24: 0.95
Acurácia para a resolução 25: 0.975
Acurácia para a resolução 26: 0.975
Acurácia para a resolução 27: 0.975
Acurácia para a resolução 28: 0.975
Acurácia para a resolução 29: 0.975
Acurácia par

### 4.4 Caso de teste: Real e Imaginario

In [18]:
dimensoes = [i for i in range(2,31)]

labels_predict_real_e_imag = []

print("Carregando...")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_real_e_imag.append(knn2(subject, dataframe))
    
print("OK!")

Carregando...
OK!


In [19]:
from sklearn.model_selection import cross_val_score

y_true = [i for i in range(0,40)]

for i, y_pred in enumerate(labels_predict_real_e_imag):
    #scores = cross_val_score(labels_predict_real_e_imag, X, y_pred, cv=1)
    print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))#scores.mean()))

Acurácia para a resolução  2: 0.575
Acurácia para a resolução  3: 0.9
Acurácia para a resolução  4: 0.95
Acurácia para a resolução  5: 0.95
Acurácia para a resolução  6: 0.95
Acurácia para a resolução  7: 0.95
Acurácia para a resolução  8: 0.95
Acurácia para a resolução  9: 0.95
Acurácia para a resolução 10: 0.95
Acurácia para a resolução 11: 0.95
Acurácia para a resolução 12: 0.975
Acurácia para a resolução 13: 0.975
Acurácia para a resolução 14: 0.975
Acurácia para a resolução 15: 0.975
Acurácia para a resolução 16: 0.975
Acurácia para a resolução 17: 0.975
Acurácia para a resolução 18: 0.975
Acurácia para a resolução 19: 0.975
Acurácia para a resolução 20: 0.975
Acurácia para a resolução 21: 0.975
Acurácia para a resolução 22: 0.975
Acurácia para a resolução 23: 0.975
Acurácia para a resolução 24: 0.975
Acurácia para a resolução 25: 0.975
Acurácia para a resolução 26: 0.975
Acurácia para a resolução 27: 0.975
Acurácia para a resolução 28: 0.975
Acurácia para a resolução 29: 0.975
Ac

## 4.5 Caso de teste: 'merge' da parte Real com Imaginária

Este caso de testes não foi possivel de ser concluído devido na falha de tentativas para conseguir calcular a distancia Euclidiana de uma matriz Numpy, como foi mencionado na função eucld().

#### TODO: Implementar este caso de testes em um momento futuro.

In [20]:
dimensoes = [i for i in range(2,31)]

labels_predict_merge_real_imag = []

print("Carregando")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_merge_real_imag.append(knn(subject, dataframe, 'rim'))
print("OK!")

Carregando


C:\Users\epita\anaconda3\envs\opencv\lib\site-packages\sklearn\metrics\_regression.py:254: RuntimeWarning: overflow encountered in square
  output_errors = np.average((y_true - y_pred) ** 2, axis=0,


ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [21]:
#y_true = [i for i in range(0,40)]

#for i, y_pred in enumerate(labels_predict_merge_real_imag):
 #   print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))

## 4.6 Caso de teste: Multiplicação da parte Real e Imaginária 

Coloquei esse teste adicional porque, enquanto tentei resolver o problema de calcular a distancia Euclidiana para o item 4.5, a forma que tratei o problema foi utilizar uma forma diferente, para fins de processamento computacional. Me deparei com um artigo no qual me lembrou a equivalencia entre a distância Euclidiana tradicional com uma variante vetorial dela, D[i,j] = (Xi -Xj)^T(Xi-Xj), onde T é a transposta da matriz. Durante esses testes eu apliquei esta multiplicação e resolvi iniciá-lo para observar o que aconteceria.

In [22]:
dimensoes = [i for i in range(2,31)]

labels_predict_real_mul_imag = []

print("Carregando")
for dimensao in dimensoes:
    subject = [cortar_imagem(dimensao, i) for i in pessoa_fft_shift]
    dataframe = [cortar_imagem(dimensao, i) for i in pessoas_fft_shift]
    labels_predict_real_mul_imag.append(knn(subject, dataframe, 'rimul'))
print("OK!")

Carregando
OK!


In [23]:
y_true = [i for i in range(0,40)]

for i, y_pred in enumerate(labels_predict_real_mul_imag):
    print("Acurácia para a resolução {:2}: {}" .format((i+2), accuracy_score(y_true, y_pred)))

Acurácia para a resolução  2: 0.925
Acurácia para a resolução  3: 0.975
Acurácia para a resolução  4: 0.975
Acurácia para a resolução  5: 0.95
Acurácia para a resolução  6: 0.975
Acurácia para a resolução  7: 0.975
Acurácia para a resolução  8: 0.95
Acurácia para a resolução  9: 0.95
Acurácia para a resolução 10: 0.95
Acurácia para a resolução 11: 0.95
Acurácia para a resolução 12: 0.95
Acurácia para a resolução 13: 0.95
Acurácia para a resolução 14: 0.95
Acurácia para a resolução 15: 0.95
Acurácia para a resolução 16: 0.95
Acurácia para a resolução 17: 0.95
Acurácia para a resolução 18: 0.95
Acurácia para a resolução 19: 0.95
Acurácia para a resolução 20: 0.95
Acurácia para a resolução 21: 0.95
Acurácia para a resolução 22: 0.95
Acurácia para a resolução 23: 0.95
Acurácia para a resolução 24: 0.95
Acurácia para a resolução 25: 0.975
Acurácia para a resolução 26: 0.975
Acurácia para a resolução 27: 0.975
Acurácia para a resolução 28: 0.975
Acurácia para a resolução 29: 0.975
Acurácia p

## 5. Conclusão

Acredito que os resultados obtidos do projeto foram satisfatíveis, com excessão do item 4.5 que se encontra não realizado e do Cross Validation, que não me ficou claro como eu faria um train_test_split de uma imagem para separar em X e y, algo que poderia ter sido resolvido anteriormente com o professor. A disciplina de Sinais e Sistemas, além de sua proposta principal, também pôde auxiliar no aumento com relação ao conhecimento que obtive através da disciplina de Inteligência Artificial e programação, assim, pude a colocar em prática novamente estas áreas, neste caso, utilizando imagens. 

![Fourier](870x489_joseph_fourier.jpg)